# Capítulo 9 — Simulación de Monte Carlo

Comentarios concretos centrados en **sintaxis**, **tips** y **errores comunes**.


In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

plt.rcParams["figure.figsize"] = (8, 4.5)
plt.rcParams["figure.dpi"] = 120


## 1) TLC con población asimétrica (chi-cuadrado)

Sintaxis:
- `stats.chi2.pdf(x, df)` -> densidad
- `stats.chi2.rvs(df, size=n, random_state=semilla)` -> muestra aleatoria
- `np.zeros(r)` -> reservar espacio para guardar resultados


In [ ]:
# Población chi-cuadrado
y = np.linspace(0, 20, num=100)
m = 1

fy = stats.chi2.pdf(y, m)
plt.plot(y, fy, color="black")
plt.grid(alpha=0.4)
plt.title(rf"PDF: Y ~ chi2({m})")
plt.show()
plt.close()

EYi = m
VYi = 2*m
EYi, VYi


In [ ]:
# Una muestra y su promedio
semilla = None   # Tip: usa un entero para reproducibilidad
n = 10

muestra = stats.chi2.rvs(m, size=n, random_state=semilla)
y_promedio = muestra.mean()

plt.scatter(np.arange(1, n+1), muestra)
plt.axhline(y_promedio, color="green")
plt.grid()
plt.show()
plt.close()

y_promedio


In [ ]:
# Monte Carlo: r promedios muestrales
campana = True
limites = None
semilla = None
n = 5
r = 1000

promedios = np.zeros(r)

for j in range(r):
    # Semilla por iteración: reproduce si semilla base no es None
    semilla2 = semilla if semilla is None else semilla + j
    muestra = stats.chi2.rvs(m, size=n, random_state=semilla2)
    promedios[j] = muestra.mean()

plt.hist(promedios, bins=15, density=True, edgecolor="black", alpha=0.9)

if campana:
    EYp = EYi
    VYp = VYi / n
    x = np.linspace(EYp - 3*np.sqrt(VYp), EYp + 3*np.sqrt(VYp), num=1000) if limites is None else np.linspace(*limites, num=1000)
    plt.plot(x, stats.norm.pdf(x, EYp, np.sqrt(VYp)), linewidth=3)

plt.grid(alpha=0.3)
plt.show()
plt.close()


In [ ]:
# Ley de grandes números: promedio vs n
semilla = None
nmax = 1000
promedios_nmax = np.zeros(nmax - 2)

for j in range(2, nmax):
    muestra = stats.chi2.rvs(m, size=j, random_state=semilla)
    promedios_nmax[j - 2] = muestra.mean()

plt.plot(promedios_nmax)
plt.axhline(EYi, color="red")
plt.grid()
plt.show()
plt.close()


## 2) Simulación de prueba t e intervalos de confianza

Tips:
- `np.std(x, ddof=1)` -> desviación estándar muestral (ERROR común: olvidar ddof)
- `ttest_1samp` devuelve p-value bilateral por defecto


In [ ]:
# Población normal (parámetros)
mu = 40
sigma = np.sqrt(100)

y = np.linspace(10, 70, num=400)
plt.plot(y, stats.norm.pdf(y, mu, sigma))
plt.grid()
plt.show()
plt.close()


In [ ]:
# Una muestra
semilla = 1
n = 10
muestra = stats.norm.rvs(mu, sigma, size=n, random_state=semilla)

promedio_m = np.mean(muestra)
desv_est_m = np.std(muestra, ddof=1)   # OJO ddof=1
error_est = desv_est_m / np.sqrt(n)

promedio_m, desv_est_m, error_est


In [ ]:
# Prueba t + valor crítico
alpha = 0.05
mu0 = mu

prueba_t = stats.ttest_1samp(muestra, popmean=mu0)
valor_crit = stats.t.ppf(1 - alpha/2, df=n - 1)

prueba_t.statistic, prueba_t.pvalue, valor_crit


In [ ]:
# Intervalo de confianza
LI = promedio_m - valor_crit * error_est
LS = promedio_m + valor_crit * error_est
(LI, LS), (mu >= LI and mu <= LS)


In [ ]:
# Monte Carlo completo: r simulaciones
mu = 40
sigma = np.sqrt(100)

semilla = 1
n = 10
alpha = 0.05
mu0 = mu
r = 20

resultados_valores_p = np.zeros(r)
resultados_error1 = np.zeros(r)
resultados_LI = np.zeros(r)
resultados_LS = np.zeros(r)
resultados_intervalo_mal = np.zeros(r)

for i in range(r):
    semilla2 = None if semilla is None else semilla + i
    muestra = stats.norm.rvs(mu, sigma, size=n, random_state=semilla2)

    promedio_m = np.mean(muestra)
    desv_est_m = np.std(muestra, ddof=1)
    error_est = desv_est_m / np.sqrt(n)

    prueba_t = stats.ttest_1samp(muestra, popmean=mu0)
    valor_crit = stats.t.ppf(1 - alpha/2, df=n - 1)

    valorp = prueba_t.pvalue
    resultados_valores_p[i] = valorp

    if valorp < alpha:
        resultados_error1[i] = 1

    LI = promedio_m - valor_crit * error_est
    LS = promedio_m + valor_crit * error_est
    resultados_LI[i] = LI
    resultados_LS[i] = LS

    if (mu < LI) or (mu > LS):
        resultados_intervalo_mal[i] = 1

tabla = pd.DataFrame({
    "Valores p": resultados_valores_p,
    "Error Tipo 1": resultados_error1,
    "Lim inferior": resultados_LI,
    "Lím superior": resultados_LS,
    "¿No contiene mu?": resultados_intervalo_mal
})
tabla


In [ ]:
# Resúmenes (proporciones)
error1_rate = resultados_error1.mean()
coverage = 1 - resultados_intervalo_mal.mean()
error1_rate, coverage


In [ ]:
# Gráfico de intervalos (rojo = no cubre mu)
r_ej = 20

plt.figure(figsize=(3, 5))
plt.xlim(10, 70)
plt.ylim(-1, r_ej)

for i in range(r_ej):
    color = "grey" if (mu > resultados_LI[i] and mu < resultados_LS[i]) else "red"
    plt.plot([resultados_LI[i], resultados_LS[i]], [i, i], color=color)

plt.axvline(mu, linestyle="--", color="green", linewidth=0.5)
plt.show()
plt.close()


## Ejercicios propuestos

1) Ejecuta la simulación con `semilla=None` y luego con `semilla=1`.

**Respuesta esperada:** con `None` cambia; con `1` se repite igual.


In [ ]:
# Escribe tu solución aquí


2) Repite la simulación del TLC con `n=5`, `n=30` y `n=100`.

**Respuesta esperada:** con n mayor, el histograma del promedio se acerca a normal.


In [ ]:
# Escribe tu solución aquí


3) En la simulación t, cambia `r` a 1000 y calcula `resultados_error1.mean()`.

**Respuesta esperada:** cercano a 0.05.


In [ ]:
# Escribe tu solución aquí


4) Calcula la cobertura: `1 - resultados_intervalo_mal.mean()` con `r=1000`.

**Respuesta esperada:** cercano a 0.95.


In [ ]:
# Escribe tu solución aquí


5) Cambia `n` de 10 a 30 y mira los intervalos.

**Respuesta esperada:** intervalos más estrechos, cobertura similar.


In [ ]:
# Escribe tu solución aquí


## Glosario

- **Monte Carlo**: repetir muchas veces un experimento aleatorio.
- **semilla**: controla reproducibilidad del azar.
- **cobertura**: proporción de IC que contienen el valor verdadero.
- **error Tipo I**: rechazar H0 siendo verdadera.
